In [1]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np
from tqdm import tqdm
# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, UpSampling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.regularizers import l2
import keras_cv
import keras_cv.layers as kcvl

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from PIL import Image
import matplotlib.gridspec as gridspec
from datetime import datetime
from sklearn.utils import shuffle
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight

# Configure plot display settings
plt.rc('font', size=14)
%matplotlib inline


E0000 00:00:1732274855.748496    8715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732274855.754988    8715 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.0


### Convnextlarge finetuning - AdamW - on training augmentation

In [2]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
tf.config.run_functions_eagerly(True)

convnext_base_model = tf.keras.applications.ConvNeXtLarge(
    include_top=False,
    weights='imagenet',
    input_shape=(96,96,3),
    classes=8,
    classifier_activation='softmax',
)
convnext_base_model.trainable = False
#tf.config.run_functions_eagerly(True)

I0000 00:00:1732274859.108142    8715 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20750 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [3]:
def build_convnext_classifier(input_shape=(96, 96, 3), seed=None):
    inputs = Input(shape=input_shape)
    x = convnext_base_model(inputs)
    x = GlobalAveragePooling2D(name="avg_pool")(x)
    x = BatchNormalization(name="batch_norm")(x)
    intermediate1 = Dense(1024, activation='swish')(x)
    dropout1 = Dropout(0.2)(intermediate1)
    intermediate2 = Dense(256, activation='swish')(dropout1)
    dropout2 = Dropout(0.2)(intermediate2)
    outputs = Dense(8, activation='softmax')(dropout2)

    model = Model(inputs=inputs, outputs=outputs, name='convnext_model')
    return model

In [4]:
data = np.load('training_set_cleaned_balancedwithrotation.npz')
X_balanced = data['images']
y_balanced = data['labels']

In [5]:
int_labels = np.array([label for label in y_balanced])
y_train_val = tfk.utils.to_categorical(int_labels)

X_train, X_val, y_train, y_val = train_test_split(X_balanced, y_train_val, random_state=42, test_size=0.1, stratify=y_train_val)

In [6]:
X_train = (X_train / 255).astype('float32') 

In [7]:
import tensorflow as tf
import keras_cv

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

rand_augment = keras_cv.layers.RandAugment(
    value_range=(0, 1),
    augmentations_per_image=4,
    magnitude=0.7,
    magnitude_stddev=0.25,
    rate=0.8,
    geometric=True,
    seed=42  
)
random_flip = tfkl.RandomFlip()

def augment_data(images, labels):
    # cast to float32 (preferred for compatibility during augmentation)
    images = tf.cast(images, tf.float32)
    labels = tf.cast(labels, tf.float32)
    inputs = {"images": images, "labels": labels}
    outputs = rand_augment(inputs)  
    outputs = random_flip(outputs)  

    return outputs['images'], outputs['labels']

def preprocess_withmultiply_convnext(images, labels):
    images = tf.cast(images * 255, tf.uint8)
    images = tf.keras.applications.convnext.preprocess_input(images)  # Preprocess the images to ResNet50V2 input format
    return images, labels

def preprocess_convnext(images, labels):
    images = tf.keras.applications.convnext.preprocess_input(images)  # Preprocess the images to ResNet50V2 input format
    return images, labels
    
train_set_aug = (
    train_dataset
    .map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)  
    .map(preprocess_withmultiply_convnext, num_parallel_calls=tf.data.AUTOTUNE)  
    .batch(32)                                              
    .prefetch(tf.data.AUTOTUNE)                             
)
test_set = (
    test_dataset
    .map(preprocess_convnext, num_parallel_calls=tf.data.AUTOTUNE) 
    .batch(32)                                            
    .prefetch(tf.data.AUTOTUNE)                    
)

In [10]:
"""
from tensorflow.keras import mixed_precision

policy = tf.keras.mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
"""

In [8]:
model = build_convnext_classifier(seed=42)

optimizer = tf.keras.optimizers.AdamW()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [10]:
def cosine_annealing_schedule(epoch, lr_start=1e-3, lr_end=1e-4, T_max=10):
    return lr_end + (lr_start - lr_end) * (1 + np.cos(np.pi * epoch / T_max)) / 2

class PlotLearning(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.metrics = {}

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for metric, value in logs.items():
                if metric not in self.metrics:
                    self.metrics[metric] = []
                self.metrics[metric].append(value)

        metrics = [x for x in logs if 'val' not in x and not x.startswith('lr')]
        f, axs = plt.subplots(1, len(metrics), figsize=(15, 5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(self.metrics[metric], label=metric)
            if 'val_' + metric in self.metrics:
                axs[i].plot(self.metrics['val_' + metric], label='val_' + metric)
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)
lr_schedule = LearningRateScheduler(cosine_annealing_schedule)
plot_callback = PlotLearning()

In [11]:
history = model.fit(
    train_set_aug,
    epochs=10,
    validation_data=test_set,
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/10


I0000 00:00:1732274910.654656    8715 cuda_dnn.cc:529] Loaded cuDNN version 90300


563/563 ━━━━━━━━━━━━━━━━━━━━ 1089s 2s/step - accuracy: 0.6176 - loss: 1.1487 - val_accuracy: 0.8990 - val_loss: 0.3267 - learning_rate: 0.0010
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1066s 2s/step - accuracy: 0.7421 - loss: 0.7471 - val_accuracy: 0.9390 - val_loss: 0.2020 - learning_rate: 9.7798e-04
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1067s 2s/step - accuracy: 0.7736 - loss: 0.6553 - val_accuracy: 0.9230 - val_loss: 0.2215 - learning_rate: 8.9414e-04
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1065s 2s/step - accuracy: 0.7963 - loss: 0.5938 - val_accuracy: 0.9405 - val_loss: 0.1720 - learning_rate: 7.3046e-04
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1064s 2s/step - accuracy: 0.8075 - loss: 0.5455 - val_accuracy: 0.9625 - val_loss: 0.1137 - learning_rate: 5.1264e-04
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1063s 2s/step - accuracy: 0.8259 - loss: 0.4976 - val_accuracy: 0.9575 - val_loss: 0.1169 - learning_rate: 3.0632e-04
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 1062s 2s/step - accuracy: 0

In [12]:
model.save("convnextlarge_offline_aug.keras")

In [1]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py convnextlarge_offline_aug.keras

  adding: model.py (deflated 49%)
  adding: convnextlarge_offline_aug.keras (deflated 7%)


#### Finetuning

In [13]:
finetuned_model = build_convnext_classifier(seed=42)
finetuned_model.load_weights('convnextlarge_offline_aug.keras')

In [14]:
b_model = finetuned_model.get_layer('convnext_large')

layers = b_model.layers

for layer in layers[-30:]:
    layer.trainable = True

In [15]:
finetuned_model.get_layer('batch_norm').trainable = False ## Suggested in tensorflow fine tuning site

In [16]:
# Display the layers and their trainable status within the 'convnext_large' submodule
print("Layers within 'convnext_large' submodule:")
for i, layer in enumerate(finetuned_model.get_layer('convnext_large').layers):
    print(i, layer.name, layer.trainable)
print("----------------------------------------------------------------------")
print("All layers in the model:")
for i, layer in enumerate(finetuned_model.layers):
    print(i, layer.name, layer.trainable)

Layers within 'convnext_large' submodule:
0 input_layer False
1 convnext_large_prestem_normalization False
2 convnext_large_stem False
3 convnext_large_stage_0_block_0_depthwise_conv False
4 convnext_large_stage_0_block_0_layernorm False
5 convnext_large_stage_0_block_0_pointwise_conv_1 False
6 convnext_large_stage_0_block_0_gelu False
7 convnext_large_stage_0_block_0_pointwise_conv_2 False
8 convnext_large_stage_0_block_0_layer_scale False
9 convnext_large_stage_0_block_0_identity False
10 convnext_large_stage_0_block_1_depthwise_conv False
11 convnext_large_stage_0_block_1_layernorm False
12 convnext_large_stage_0_block_1_pointwise_conv_1 False
13 convnext_large_stage_0_block_1_gelu False
14 convnext_large_stage_0_block_1_pointwise_conv_2 False
15 convnext_large_stage_0_block_1_layer_scale False
16 convnext_large_stage_0_block_1_identity False
17 convnext_large_stage_0_block_2_depthwise_conv False
18 convnext_large_stage_0_block_2_layernorm False
19 convnext_large_stage_0_block_2_poi

In [17]:
def cosine_annealing_schedule(epoch, lr_start=3e-5, lr_end=1e-5, T_max=2):
    return lr_end + (lr_start - lr_end) * (1 + np.cos(np.pi * epoch / T_max)) / 2

In [18]:
# Callbacks for training
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2)
lr_schedule = ReduceLROnPlateau(patience=1, factor=0.5, monitor="val_loss")
lr_schedule = LearningRateScheduler(cosine_annealing_schedule)

In [19]:
#optimizer = tf.keras.optimizers.Lion(learning_rate=5e-5, weight_decay=0.01)
optimizer = tf.keras.optimizers.AdamW(learning_rate=3e-5, weight_decay=0.01)
finetuned_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [20]:
# Assuming the model is defined elsewhere
history = finetuned_model.fit(
    train_set_aug,
    epochs=3,
    validation_data=test_set,
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 1143s 2s/step - accuracy: 0.8502 - loss: 0.4257 - val_accuracy: 0.9650 - val_loss: 0.1007 - learning_rate: 3.0000e-05
Epoch 2/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 1142s 2s/step - accuracy: 0.8708 - loss: 0.3725 - val_accuracy: 0.9705 - val_loss: 0.0916 - learning_rate: 2.0000e-05
Epoch 3/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 1148s 2s/step - accuracy: 0.8752 - loss: 0.3693 - val_accuracy: 0.9700 - val_loss: 0.1003 - learning_rate: 1.0000e-05


In [21]:
# Save the final model
finetuned_model.save("convnextlarge_offline_aug_ft.keras")

In [2]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py Convnextlargeaugft.keras

  adding: model.py (deflated 49%)
  adding: Convnextlargeaugft.keras (deflated 7%)


### Convnext Fine Tuning - Augmentation as a model layer - AdamW

In [2]:
convnext_base_model = tf.keras.applications.ConvNeXtXLarge(
    include_top=False,
    weights='imagenet',
    input_shape=(96,96,3),
    classes=8,
    classifier_activation='softmax',
)
convnext_base_model.trainable = False
tf.config.run_functions_eagerly(True)

I0000 00:00:1732182471.657207  459371 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20750 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [3]:
"""
for layer in convnext_base_model.layers[-50:]:  # Fine-tune last 50 layers
    layer.trainable = True
"""

'\nfor layer in convnext_base_model.layers[-50:]:  # Fine-tune last 50 layers\n    layer.trainable = True\n'

In [3]:
# Register the custom augmentation layer for serialization
from tensorflow.keras.applications import ConvNeXtLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization
import tensorflow.keras.layers as tfkl

@tf.keras.utils.register_keras_serializable(package="Custom")
class ImageAugmentationLayer(tfkl.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)  
        self.seed = seed
        self.rand_augment = kcvl.RandAugment(
            value_range=(0, 1),
            augmentations_per_image=3,
            magnitude=0.5,
            magnitude_stddev=0.2,
            rate=0.9090909090909091,
            geometric=True,
            seed=seed
        )
        self.random_flip = tfkl.RandomFlip()
        self.random_zoom = tfkl.RandomZoom(height_factor=(-0.05, 0.1), width_factor=(-0.05, 0.1))
        self.random_rotation = tfkl.RandomRotation(factor=0.5)

    def call(self, inputs, training=True):
        if training:
            x = self.rand_augment(inputs)
            x = self.random_flip(x)
            x = self.random_zoom(x)
            x = self.random_rotation(x)
            return x
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"seed": self.seed})
        return config

# Define a custom layer for the `tf.cast` operation
@tf.keras.utils.register_keras_serializable(package="Custom")
class CastToUint8Layer(tfkl.Layer):
    def call(self, inputs):
        return tf.cast(inputs * 255, tf.uint8)

# Define the model using the custom augmentation layer
def build_convnext_with_augmentation(input_shape=(96, 96, 3), seed=None):
    inputs = Input(shape=input_shape)

    augmented = ImageAugmentationLayer(seed=seed)(inputs)
    augmented_255 = CastToUint8Layer()(augmented)
    
    x1 = convnext_base_model(augmented_255)  
    x2 = tfkl.GlobalAveragePooling2D(name="avg_pool")(x1)
    x3 = tfkl.BatchNormalization(name="batch_norm")(x2)
    intermediate1 = Dense(1024, activation='swish')(x3)
    dropout1 = Dropout(0.2)(intermediate1)
    intermediate2 = Dense(256, activation='swish')(dropout1)
    outputs = tfkl.Dense(8, activation='softmax')(intermediate2)

    model = Model(inputs=inputs, outputs=outputs, name='convnext_augmented_model')
    return model

@tf.keras.utils.register_keras_serializable(package="Custom")
class ImageAugmentationLayerWithAugMix(tfkl.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.seed = seed
        self.rand_augment = kcvl.RandAugment(
            value_range=(0, 1),
            augmentations_per_image=2,
            magnitude=0.4,
            magnitude_stddev=0.15,
            seed=seed
        )
        self.augmix = kcvl.AugMix(
            value_range=(0, 1),
            severity=0.15,
            seed=seed
        )
        self.random_flip = tfkl.RandomFlip()
        self.random_zoom = tfkl.RandomZoom(height_factor=(-0.15, 0.2), width_factor=(-0.15, 0.2))
        self.random_rotation = tfkl.RandomRotation(factor=0.5)

    def call(self, inputs, training=True):
        if training:
            x = self.augmix(inputs)
            x = self.rand_augment(x)
            x = self.random_flip(x)
            x = self.random_zoom(x)
            x = self.random_rotation(x)
            return x
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"seed": self.seed})
        return config

def build_convnext_with_augmix_augmentation(input_shape=(96, 96, 3), seed=None):
    inputs = Input(shape=input_shape)
    augmented = ImageAugmentationLayerWithAugMix(seed=seed)(inputs)
    # Convert to uint8 [0, 255] range after augmentation
    augmented_255 = CastToUint8Layer()(augmented)
    features = convnext_base_model(augmented_255)
    pooled = GlobalAveragePooling2D(name="avg_pool")(features)
    normalized = BatchNormalization()(pooled)
    x = Dense(1024, activation='swish')(normalized)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='swish')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(8, activation='softmax')(x)
    return Model(inputs, outputs, name='convnext_augmented_model')

In [4]:
from tensorflow.keras.applications import convnext

data = np.load('training_set_cleaned_balancedwithrotation.npz')
X_balanced = data['images']
y_balanced = data['labels']

In [5]:
X_train_val = (X_balanced / 255).astype('float32')
int_labels = np.array([label for label in y_balanced])
y_train_val = tfk.utils.to_categorical(int_labels)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.1, stratify=y_train_val)

In [7]:
model = build_convnext_with_augmentation(seed=42)
#model = build_convnext_with_augmix_augmentation(seed=42)

In [8]:
#optimizer = tf.keras.optimizers.Lion(learning_rate=1e-3, weight_decay=0.004)
optimizer = tf.keras.optimizers.AdamW()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [9]:
def cosine_annealing_schedule(epoch, lr_start=1e-3, lr_end=3e-4, T_max=5):
    return lr_end + (lr_start - lr_end) * (1 + np.cos(np.pi * epoch / T_max)) / 2

In [10]:
from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

class PlotLearning(Callback):
    def on_train_begin(self, logs=None):
        # Initializes a dictionary to store training metrics
        self.metrics = {}
        if logs is not None:
            for metric in logs:
                self.metrics[metric] = []

    def on_epoch_end(self, epoch, logs=None):
        # Store metrics
        if logs is not None:
            for metric in logs:
                if metric in self.metrics:
                    self.metrics[metric].append(logs.get(metric))
                else:
                    self.metrics[metric] = [logs.get(metric)]

        # Plotting
        metrics = [x for x in logs if 'val' not in x and not x.startswith('lr')]

        # Set up subplots
        f, axs = plt.subplots(1, len(metrics), figsize=(15, 5))
        clear_output(wait=True)

        # Convert axs to a numpy array for indexing
        if not isinstance(axs, np.ndarray):
            axs = np.array([axs])

        # Plot learning curves
        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2),
                        self.metrics[metric],
                        label=metric)
            if 'val_' + metric in self.metrics:
                axs[i].plot(range(1, epoch + 2),
                            self.metrics['val_' + metric],
                            label='val_' + metric)

            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2)
lr_schedule = ReduceLROnPlateau(patience=1, factor=0.5, monitor="val_loss")
lr_schedule = LearningRateScheduler(cosine_annealing_schedule)
plot_callback = PlotLearning()

In [11]:
class_indices = np.argmax(y_train, axis=1)

# Use np.unique to calculate class frequencies
unique_classes, counts = np.unique(class_indices, return_counts=True)

# Print the class frequencies
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} samples")

Class 0: 2250 samples
Class 1: 2250 samples
Class 2: 2250 samples
Class 3: 2250 samples
Class 4: 2250 samples
Class 5: 2250 samples
Class 6: 2250 samples
Class 7: 2250 samples


In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=5,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_schedule, plot_callback]
)

Epoch 1/5


I0000 00:00:1732149110.373597  367821 cuda_dnn.cc:529] Loaded cuDNN version 90300


 18/563 ━━━━━━━━━━━━━━━━━━━━ 41:42 5s/step - accuracy: 0.3620 - loss: 1.9208

In [ ]:
model.summary()

In [ ]:
model.save('ConvNeXtXLarge_augmented_final.keras')

In [ ]:
print("done")

In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py ConvNeXtXLarge_augmented_final.keras

#### Finetune

In [6]:
# Create a new instance of CustomModel for fine-tuning
finetuned_model = build_convnext_with_augmentation(seed=42)

# Load the weights for the model
finetuned_model.load_weights('ConvNeXtXLarge_augmented_final.keras')

In [19]:
b_model = finetuned_model.get_layer('convnext_xlarge')

layers = b_model.layers

for layer in layers[-30:]:
    layer.trainable = True

Last 30 layers of convnext_xlarge are now trainable:
Layer: convnext_xlarge_stage_2_block_26_depthwise_conv, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_layernorm, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_pointwise_conv_1, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_gelu, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_pointwise_conv_2, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_layer_scale, Trainable: True
Layer: convnext_xlarge_stage_2_block_26_identity, Trainable: True
Layer: convnext_xlarge_downsampling_block_2, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_depthwise_conv, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_layernorm, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_pointwise_conv_1, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_gelu, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_pointwise_conv_2, Trainable: True
Layer: convnext_xlarge_stage_3_block_0_layer_scale, Traina

In [16]:
finetuned_model.get_layer('batch_norm').trainable = False

In [20]:
# Display the layers and their trainable status within the 'convnext_large' submodule
print("Layers within 'convnext_large' submodule:")
for i, layer in enumerate(finetuned_model.get_layer('convnext_xlarge').layers):
    print(i, layer.name, layer.trainable)

print("\n")

# Display all layers in the entire model and their trainable status
print("All layers in the model:")
for i, layer in enumerate(finetuned_model.layers):
    print(i, layer.name, layer.trainable)

Layers within 'convnext_large' submodule:
0 input_layer False
1 convnext_xlarge_prestem_normalization False
2 convnext_xlarge_stem False
3 convnext_xlarge_stage_0_block_0_depthwise_conv False
4 convnext_xlarge_stage_0_block_0_layernorm False
5 convnext_xlarge_stage_0_block_0_pointwise_conv_1 False
6 convnext_xlarge_stage_0_block_0_gelu False
7 convnext_xlarge_stage_0_block_0_pointwise_conv_2 False
8 convnext_xlarge_stage_0_block_0_layer_scale False
9 convnext_xlarge_stage_0_block_0_identity False
10 convnext_xlarge_stage_0_block_1_depthwise_conv False
11 convnext_xlarge_stage_0_block_1_layernorm False
12 convnext_xlarge_stage_0_block_1_pointwise_conv_1 False
13 convnext_xlarge_stage_0_block_1_gelu False
14 convnext_xlarge_stage_0_block_1_pointwise_conv_2 False
15 convnext_xlarge_stage_0_block_1_layer_scale False
16 convnext_xlarge_stage_0_block_1_identity False
17 convnext_xlarge_stage_0_block_2_depthwise_conv False
18 convnext_xlarge_stage_0_block_2_layernorm False
19 convnext_xlarge_

In [21]:
def cosine_annealing_schedule(epoch, lr_start=3e-5, lr_end=1e-5, T_max=2):
    return lr_end + (lr_start - lr_end) * (1 + np.cos(np.pi * epoch / T_max)) / 2

In [22]:
# Callbacks for training
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2)
lr_schedule = ReduceLROnPlateau(patience=1, factor=0.5, monitor="val_loss")
lr_schedule = LearningRateScheduler(cosine_annealing_schedule)

In [23]:
#optimizer = tf.keras.optimizers.Lion(learning_rate=5e-5, weight_decay=0.01)
optimizer = tf.keras.optimizers.AdamW(learning_rate=3e-5, weight_decay=0.01)
finetuned_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [24]:
ft_history = finetuned_model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=2,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/2


I0000 00:00:1732183047.541444  459371 cuda_dnn.cc:529] Loaded cuDNN version 90300


372/563 ━━━━━━━━━━━━━━━━━━━━ 14:39 5s/step - accuracy: 0.9034 - loss: 0.2836

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9448eb0290>>
Traceback (most recent call last):
  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [25]:
finetuned_model.save("ConvNeXtXLarge_augmented_final_ft.keras")

In [26]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py ConvNeXtXLarge_augmented_final_ft.keras

  adding: model.py (deflated 63%)
  adding: ConvNeXtXLarge_augmented_final_ft.keras (deflated 7%)


### AdamW - ResNetV2

In [15]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
tf.config.run_functions_eagerly(True)

resnet_base_model = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(96,96,3),
    classes=8
)
resnet_base_model.trainable = False  

@tf.keras.utils.register_keras_serializable(package="Custom")
class ImageAugmentationLayer(tfkl.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.seed = seed
        self.rand_augment = kcvl.RandAugment(
            value_range=(0, 1),
            augmentations_per_image=3,
            magnitude=0.6,
            magnitude_stddev=0.2,
            rate=0.9090909090909091,
            geometric=True,
            seed=seed
        )
        self.random_flip = tfkl.RandomFlip()
        self.random_zoom = tfkl.RandomZoom(height_factor=(-0.05, 0.1), width_factor=(-0.05, 0.1))
        self.random_rotation = tfkl.RandomRotation(factor=0.5)

    def call(self, inputs, training=True):
        if training:
            x = self.rand_augment(inputs)
            x = self.random_flip(x)
            x = self.random_zoom(x)
            x = self.random_rotation(x)
            return x
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"seed": self.seed})
        return config

@tf.keras.utils.register_keras_serializable(package="Custom")
class CastToUint8Layer(tfkl.Layer):
    def call(self, inputs):
        return tf.cast(inputs * 255, tf.uint8)


def build_inceptionresnet_with_augmentation(input_shape=(96, 96, 3), seed=None):
    inputs = Input(shape=input_shape)
    augmented = ImageAugmentationLayer(seed=seed)(inputs)
    augmented_255 = CastToUint8Layer()(augmented)

    x = resnet_base_model(augmented_255)
    x = GlobalAveragePooling2D(name="avg_pool")(x)
    x = BatchNormalization(name="batch_norm")(x)
    intermediate1 = Dense(1024, activation='swish')(x)
    dropout1 = Dropout(0.2)(intermediate1)
    intermediate2 = Dense(256, activation='swish')(dropout1)
    outputs = Dense(8, activation='softmax')(intermediate2)

    model = Model(inputs=inputs, outputs=outputs, name='inceptionresnet_augmented_model')
    return model



In [16]:
data = np.load('training_set_cleaned_balancedwithrotation.npz')
X_balanced = data['images']
y_balanced = data['labels']

X_train_val = (X_balanced / 255).astype('float32')
int_labels = np.array([label for label in y_balanced])
y_train_val = tfk.utils.to_categorical(int_labels)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.1, stratify=y_train_val)


In [26]:
X_train.shape

(18000, 96, 96, 3)

In [25]:
y_train.shape

(18000, 8)

In [17]:
model = build_inceptionresnet_with_augmentation(seed=42)

optimizer = tf.keras.optimizers.AdamW()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/tensorflow/python/ops/while_loop.py", line 488, in while_loop
    loop_vars = body(*loop_vars)  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/tensorflow/python/ops/while_loop.py", line 479, in <lambda>
    body = lambda i, lv: (i + 1, orig_body(*lv))  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/tensorflow/python/ops/map_fn.py", line 495, in compute
    return (i + 1, tas)  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/tensorflow/python/ops/map_fn.py", line 492, in <listcomp>
    tas = [  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


In [18]:
def cosine_annealing_schedule(epoch, lr_start=1e-3, lr_end=3e-4, T_max=20):
    return lr_end + (lr_start - lr_end) * (1 + np.cos(np.pi * epoch / T_max)) / 2

class PlotLearning(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.metrics = {}

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for metric, value in logs.items():
                if metric not in self.metrics:
                    self.metrics[metric] = []
                self.metrics[metric].append(value)

        metrics = [x for x in logs if 'val' not in x and not x.startswith('lr')]
        f, axs = plt.subplots(1, len(metrics), figsize=(15, 5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(self.metrics[metric], label=metric)
            if 'val_' + metric in self.metrics:
                axs[i].plot(self.metrics['val_' + metric], label='val_' + metric)
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)
lr_schedule = LearningRateScheduler(cosine_annealing_schedule)
plot_callback = PlotLearning()

In [19]:
# Train the model
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_schedule, plot_callback]
)

Epoch 1/20
404/563 ━━━━━━━━━━━━━━━━━━━━ 5:04 2s/step - accuracy: 0.2810 - loss: 1.9545

KeyboardInterrupt: 

In [ ]:
model.save('InceptionResNetV2_augmented_final.keras')

### Lion - EfficientNet - Offline Aug

In [2]:
data = np.load('balanced_dataset_leaves_augmented_augmix_48000.npz')
X = data['images']
y = data['labels']

In [3]:
data = np.load('balanced_dataset_leaves_augmented_augmix_48000_v2.npz')
X_2 = data['images']
y_2 = data['labels']
X = np.concatenate((X, X_2), axis=0)
y = np.concatenate((y, y_2), axis=0)

In [4]:
data = np.load('balanced_dataset_leaves_augmented.npz')
X_3 = data['images']
y_3 = data['labels']
X = np.concatenate((X, X_3), axis=0)
y = np.concatenate((y, y_3), axis=0)

In [5]:
data = np.load('training_set_shreks_removed.npz')
X_4 = data['images']
y_4 = data['labels']
X = np.concatenate((X, X_4), axis=0)
y = np.concatenate((y, y_4), axis=0)

In [11]:
del X_2,X_3,X_4, y_2, y_3, y_4

In [6]:
X.shape

(131959, 96, 96, 3)

In [9]:
# Flatten X for easier comparison and find unique indices
X_flat = X.reshape(X.shape[0], -1)  # Flatten each image into a 1D array
unique_indices = np.unique(X_flat, axis=0, return_index=True)[1]

# Sort indices to maintain order
unique_indices_sorted = np.sort(unique_indices)

# Use unique indices to filter X and y
X_unique = X[unique_indices_sorted]
y_unique = y[unique_indices_sorted]

In [10]:
X_unique.shape

(96074, 96, 96, 3)

In [13]:
y_unique.shape

(96074,)

In [14]:
X = X_unique
y = y_unique

In [18]:
# Split dataset into training, validation, and testing
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import EfficientNetB7, EfficientNetB0, EfficientNetV2L
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.efficientnet import preprocess_input

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# One-hot encode labels
num_classes = len(np.unique(y))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Normalize dataset (if not already normalized)
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

# Data augmentation pipeline
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
    ],
    name="data_augmentation",
)

# Load EfficientNetB0 model (with ImageNet weights)
base_model = EfficientNetV2L(weights="imagenet", include_top=False, input_shape=(96, 96, 3), classes = 8)
base_model.trainable = False

# Build the model
def build_efficientnet_without_augmentation(input_shape=(96, 96, 3)):
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation=tf.keras.activations.swish)(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = layers.Dense(64, activation=tf.keras.activations.swish)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs, name='efficientnet_model')
    return model

# Compile the model
model = build_efficientnet_without_augmentation()

# Compile the model with AdamW
initial_lr = 1e-3
weight_decay = 1e-2
optimizer = AdamW(learning_rate=initial_lr, weight_decay=weight_decay)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Callbacks for generalization
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor="val_loss"),
    ReduceLROnPlateau(patience=1, factor=0.5, monitor="val_loss"),
]

# Train the top layers
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
)

473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/50


I0000 00:00:1731932684.069781   27865 service.cc:148] XLA service 0x7ff4bc0019b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731932684.070289   27865 service.cc:156]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
I0000 00:00:1731932691.197670   27865 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/1352 ━━━━━━━━━━━━━━━━━━━━ 34:00:17 91s/step - accuracy: 0.1094 - loss: 2.2456

I0000 00:00:1731932733.055778   27865 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1352/1352 ━━━━━━━━━━━━━━━━━━━━ 222s 97ms/step - accuracy: 0.5761 - loss: 1.1977 - val_accuracy: 0.7469 - val_loss: 0.6861 - learning_rate: 0.0010
Epoch 2/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 47ms/step - accuracy: 0.7007 - loss: 0.8500 - val_accuracy: 0.7644 - val_loss: 0.6650 - learning_rate: 0.0010
Epoch 3/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 46ms/step - accuracy: 0.7224 - loss: 0.7838 - val_accuracy: 0.7723 - val_loss: 0.6261 - learning_rate: 0.0010
Epoch 4/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 62s 46ms/step - accuracy: 0.7265 - loss: 0.7667 - val_accuracy: 0.7841 - val_loss: 0.5882 - learning_rate: 0.0010
Epoch 5/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 47ms/step - accuracy: 0.7502 - loss: 0.7113 - val_accuracy: 0.7937 - val_loss: 0.5691 - learning_rate: 0.0010
Epoch 6/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 64s 48ms/step - accuracy: 0.7582 - loss: 0.6853 - val_accuracy: 0.7950 - val_loss: 0.5599 - learning_rate: 0.0010
Epoch 7/50
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 47ms/step - accuracy: 0.7608

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ff86dd28190>>
Traceback (most recent call last):
  File "/opt/conda/envs/myenv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [19]:
# Save the final model
model.save("efficientnetV2L.keras")

In [20]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py efficientnetV2L.keras

  adding: model.py (deflated 50%)
  adding: efficientnetV2L.keras (deflated 11%)


In [21]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-50:]:  
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5, weight_decay=weight_decay),  
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Fine-tune the model
fine_tune_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 248s 110ms/step - accuracy: 0.7652 - loss: 0.6784 - val_accuracy: 0.8389 - val_loss: 0.4531 - learning_rate: 1.0000e-05
Epoch 2/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 78s 57ms/step - accuracy: 0.8402 - loss: 0.4511 - val_accuracy: 0.8522 - val_loss: 0.4171 - learning_rate: 1.0000e-05
Epoch 3/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 77s 57ms/step - accuracy: 0.8584 - loss: 0.4002 - val_accuracy: 0.8566 - val_loss: 0.4068 - learning_rate: 1.0000e-05
Epoch 4/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 77s 57ms/step - accuracy: 0.8702 - loss: 0.3639 - val_accuracy: 0.8643 - val_loss: 0.3933 - learning_rate: 1.0000e-05
Epoch 5/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 77s 57ms/step - accuracy: 0.8764 - loss: 0.3429 - val_accuracy: 0.8684 - val_loss: 0.3850 - learning_rate: 1.0000e-05
Epoch 6/30
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 77s 57ms/step - accuracy: 0.8849 - loss: 0.3230 - val_accuracy: 0.8697 - val_loss: 0.3788 - learning_rate: 1.0000e-05
Epoch 7/30
1352/1352 ━━━━━━━━━━━━━━━━━

In [22]:
# Save the final model
model.save("efficientnetV2L_unfreezed.keras")

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
eff_net = load_model("final_efficientnetB7.keras")
sample_image = np.expand_dims(X_test[1000], axis=0)  # Shape becomes (1, height, width, channels)
predictions = eff_net.predict(sample_image)
if len(predictions.shape) == 2:
    predictions = np.argmax(predictions, axis=1)
predictions

In [ ]:
plt.imshow(X_test[1009])  # Display the image
plt.axis('off')  # Optionally hide the axis
plt.show()

In [24]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'
!zip {filename} model.py efficientnetV2L_unfreezed.keras

  adding: model.py (deflated 50%)
  adding: efficientnetV2L_unfreezed.keras (deflated 10%)


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import keras_cv.layers as kcvl

@tf.keras.utils.register_keras_serializable(package="Custom")
class ImageAugmentationLayer(tfkl.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)  # Pass any unexpected args to the parent class
        self.seed = seed
        self.rand_augment = kcvl.RandAugment(
            value_range=(0, 1),
            augmentations_per_image=2,
            magnitude=0.5,
            magnitude_stddev=0.20,
            rate=0.9090909090909091,
            geometric=True,
            seed=seed
        )
        self.random_flip = tfkl.RandomFlip()
        self.random_zoom = tfkl.RandomZoom(height_factor=(-0.15, 0.2), width_factor=(-0.15, 0.2))
        self.random_rotation = tfkl.RandomRotation(factor=0.5)

    def call(self, inputs, training=True):
        if training:
            x = self.rand_augment(inputs)
            x = self.random_flip(x)
            x = self.random_zoom(x)
            x = self.random_rotation(x)
            return x
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"seed": self.seed})
        return config

# Define a custom layer for the `tf.cast` operation
@tf.keras.utils.register_keras_serializable(package="Custom")
class CastToUint8Layer(tfkl.Layer):
    def call(self, inputs):
        return tf.cast(inputs * 255, tf.uint8)
        
class Model:
    def __init__(self):
        self.neural_network_augmentation = tfk.models.load_model("ConvNeXtLargeunfreezedweights.keras")
        self.neural_network = tfk.Model(inputs= self.neural_network_augmentation.layers[2].input, outputs= self.neural_network_augmentation.output)
    def predict(self, X):
        X = X / 255.0
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

model = Model()

In [7]:
model.neural_network.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ keras_tensor_1293CLONE          │ (None, 96, 96, 3)      │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_to_uint8_layer             │ (None, 96, 96, 3)      │             0 │
│ (CastToUint8Layer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convnext_large (Functional)     │ (None, 3, 3, 1536)     │   196,230,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ avg_pool                        │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1536)           │         6,144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,573,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 198,074,824 (755.60 MB)

 Trainable params: 68,219,656 (260.24 MB)

 Non-trainable params: 129,855,168 (495.36 MB)

In [17]:
test_loss, test_accuracy = model.evaluate(X_val, y_val, verbose=1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 373ms/step - accuracy: 0.9690 - loss: 0.1111


In [18]:
history

###  Test with another blood cell dataset

In [22]:
import os
import cv2
import numpy as np

# Define the folder containing the images and the label mapping
data_folder = "./PBC_dataset_normal_DIB"
label_mapping = {
    "basophil": 0,
    "eosinophil": 1,
    "erythroblast": 2,
    "ig": 3,
    "lymphocyte": 4,
    "monocyte": 5,
    "neutrophil": 6,
    "platelet": 7,
}

# Initialize lists to store image data and labels
images = []
labels = []

# Read the images and labels
for class_name, label in label_mapping.items():
    class_folder = os.path.join(data_folder, class_name)
    if not os.path.isdir(class_folder):
        continue
    
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        # Read the image (adjust flag to 0 for grayscale or 1 for color images)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            continue  # Skip unreadable files
        
        # Resize the image to a fixed size (e.g., 96x96)
        img_resized = cv2.resize(img, (96, 96))
        images.append(img_resized)
        labels.append(label)

# Convert lists to numpy arrays
X_test1 = np.array(images, dtype=np.float32) / 255.0  # Normalize pixel values to [0, 1]
y_test1 = np.array(labels, dtype=np.int32)



Dataset created with 17092 samples.


NameError: name 'y' is not defined

In [23]:

print(f"Dataset created with {X_test1.shape[0]} samples.")
print(f"Image shape: {X_test1.shape[1:]}, Labels shape: {y_test1.shape}")

Dataset created with 17092 samples.
Image shape: (96, 96, 3), Labels shape: (17092,)


In [26]:
num_classes = len(label_mapping)
y_test1_onehot = to_categorical(y_test1, num_classes=num_classes)

In [29]:
class_indices = np.argmax(y_test1_onehot, axis=1)

# Use np.unique to calculate class frequencies
unique_classes, counts = np.unique(class_indices, return_counts=True)

# Print the class frequencies
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} samples")

Class 0: 1218 samples
Class 1: 3117 samples
Class 2: 1551 samples
Class 3: 2895 samples
Class 4: 1214 samples
Class 5: 1420 samples
Class 6: 3329 samples
Class 7: 2348 samples


In [27]:
test_loss, test_accuracy = model.evaluate(X_test1, y_test1_onehot, verbose=1)

535/535 ━━━━━━━━━━━━━━━━━━━━ 199s 371ms/step - accuracy: 0.3649 - loss: 2.7882


In [28]:
import numpy as np

# Example datasets (replace these with your actual datasets)
# X_test1 and X_train_val should be numpy arrays of shape (num_samples, height, width, channels)
# E.g., X_test1 = np.array([...]), X_train_val = np.array([...])

# Create hash sets for both datasets
def hash_images(image_dataset):
    return set(hash(img.tobytes()) for img in image_dataset)

hashes_test1 = hash_images(X_test1)
hashes_train_val = hash_images(X_train_val)

# Find the intersection
common_hashes = hashes_test1.intersection(hashes_train_val)

# Output results
if common_hashes:
    print(f"Found {len(common_hashes)} common images between the datasets.")
else:
    print("No common images found between the datasets.")


No common images found between the datasets.
